## Steps for Format Data

1. Split data on SID
1. For each student, order questions by time
1. For each student, split questions by concept
1. Convert into a sequence of 1's and 0's
1. Combine all concept sequences for all sequences



In [33]:
import pandas as pd
import numpy as np
import csv
import scipy.io as sio

In [70]:
# Some rows of KDD data contain multiple concepts. Split these into multiple rows, one per concept.
# Additionally, remove unlabeled rows (rows with no labeled concept).

infile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train.txt"
outfile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_clean.txt"

with open(infile_path,'r') as infile, open(outfile_path, 'w') as outfile:
    csvreader = csv.reader(infile, dialect="excel-tab")
    csvwriter = csv.writer(outfile, dialect = "excel-tab")
    for row in csvreader:
        if not row[19]:
            continue
        # Write one row per concept in 'kc_ktracedskills' column 19
        concepts = row[19].split('~~')
        opportunities = row[20].split('~~')
        for c, o in zip(concepts, opportunities):
            temprow = row
            temprow[19] = c
            temprow[20] = o
            csvwriter.writerow(temprow)

In [71]:
# Create a Dictionary Linking Student ID to Number of Questions and Concept ID to Number of Questions

def get_kdd_info(file_path):
    info = dict()
    student_count = dict()
    concept_count = dict()
    with open(file_path,'r') as data_file:
        csvreader = csv.reader(data_file, dialect="excel-tab")
        for row in csvreader:
            studentid = row[1]
            conceptid = row[19]
            if studentid not in student_count:
                student_count[studentid] = 1
            else:
                student_count[studentid] += 1
            if conceptid not in concept_count:
                concept_count[conceptid] = 1
            else:
                concept_count[conceptid] += 1
    info['student_count'] = student_count
    info['concept_count'] = concept_count
    return info

file_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_clean.txt"

kdd_info = get_kdd_info(file_path)
#print kdd_info['concept_count']
#print kdd_info['student_count']

In [6]:
# Subsetting the data to 50 random students
def get_random_sample(kdd_info, sample_size):
    sid_list = []
    for sid, numqs in kdd_info['student_count'].iteritems():
        sid_list.append(sid)

    return np.random.choice(sid_list, sample_size, replace=False)
    
random_students = get_random_sample(student_ids, 50)

In [72]:
def get_top_n_concepts(concept_dict, n, clean=True):
    c = concept_dict
    s = sorted(c, key=c.get, reverse=True)
    #s = sorted(c.items(), key=lambda x:x[1], reverse=True)
    top_items = []
    for entry in s:
        if clean:
            if 'enter' in entry.lower() or not entry:
                continue
        top_items.append(entry)
        if len(top_items) >= n:
            break
    return top_items
    
top_concepts = get_top_n_concepts(kdd_info['concept_count'], 20)

In [74]:
# Make a modified subset of the KDD Cup data file that has data from our random sample of students.
# This is tailored for the KDD Cup data; we will have to modify this when using other data sources.
# The CSV file is nearly identical to input file, with two changes:
# 1. Some questions are assigned multiple concepts; we split these into one row per concept
# 2. We only keep data from a random sample of students

infile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_clean.txt"
outfile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_topConcepts.txt"

total_rows = 0
written_rows = 0
with open(infile_path,'r') as infile, open(outfile_path, 'w') as outfile:
    csvreader = csv.reader(infile, dialect="excel-tab")
    csvwriter = csv.writer(outfile, dialect = "excel-tab")
    for row in csvreader:
        concept_id = row[19]
        total_rows += 1
        if concept_id not in top_concepts:
            continue
        csvwriter.writerow(row)
        written_rows += 1
        
print total_rows
print written_rows

12536294
4122077


In [75]:
top_concepts_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_topConcepts.txt"
top_concept_info = get_kdd_info(top_concepts_path)

20
['Calculate with 2 digit numerator-1', 'Identify number of recipients-1', 'Calculate internal product-1', 'Calculate difference digit from 2 digits-1', 'Calculate difference digit -- no borrow-1', 'Identify number as common factor-1', 'Calculate sum digit from 2 digits-1', 'Rewrite fraction with common denominator-1', 'Identify number as common multiple-1', 'Calculate partial product -- carry out-1', 'Find Y, any form-1', 'Calculate internal difference-1', 'List consecutive multiples of a number-1', 'Identify number of items-1', 'Identify that a fraction can/cannot be simplified-1', 'Calculate sum digit -- no carry-1', 'Calculate partial product -- no carry-1', 'Calculate partial product -- carry in-1', 'Calculate partial product -- carry in and out-1', 'List factor of large number-1']
['Calculate sum digit -- no carry-1', 'Identify number as common factor-1', 'Calculate partial product -- no carry-1', 'Calculate difference digit from 2 digits-1', 'List consecutive multiples of a nu

In [9]:
# Subset data to ensure we have enough data per concept
# For each concept, find all students who answer qs on that concept, drop students who answer very few questions,
# Count remaining students, if remaining students are very few, drop that concept

def threshold_data(rawdata, min_answer_count, min_student_count):
    # Read the CSV file into a pandas dataframe. This allows us to parse the date column and sort by date, 
    # which is important for ensuring that the questions are in order.
    # We then split the data into one pandas dataframe per student using the pandas groupby function
    kdd_col_names = ['row','student_id','problem_hierarchy', 'problem_name', 'problem_view','step_name','step_start_time','first_transaction_time','correct_transaction_time','step_end_time', 'step_duration','correct_step_duration', 'error_step_duration','correct_first_attempt', 'incorrects', 'hints', 'corrects', 'kc_subskills', 'opportunity_subskills', 'kc_ktracedskills', 'opportunity_ktracedskills']
    pd_allstudents = pd.read_csv(outfile_path, sep='\t', names=kdd_col_names, parse_dates=[6,7,8,9], infer_datetime_format=True)
    pd_allstudents = pd_allstudents.sort_values('step_start_time') # Sort by datetime
    
    grouped_by_concept = pd_allstudents.groupby(['kc_ktracedskills'])

    concepts_to_keep = []
    for name,group in grouped_by_concept:
        qs_per_student = group['student_id'].value_counts()
        
        # Find students whose number of questions answered is below answer_threshold
        to_keep = []
        for k,v in qs_per_student.iteritems():
            if v >= min_answer_count:
                to_keep.append(k)

        group = group[group.student_id.isin(to_keep)]
        
        if len(set(to_keep)) >= min_student_count:
            concepts_to_keep.append(name)
            
    # Merge updated concepts
    subset_df = None
    first_time = True
    running_sum = 0
    for c in concepts_to_keep:
        df = grouped_by_concept.get_group(c)
        running_sum += df.shape[0]
        if first_time:
            subset_df = df
            first_time = False
            continue
        subset_df = pd.concat([subset_df, df])
    return subset_df

In [14]:
outfile_path = "/Volumes/Slim2TB/classes/cs229/project/data/KDD Cup/kddcup_challenge/bridge_to_algebra_2008_2009_train_clean.txt"
cleaned_data = threshold_data(outfile_path, 10, 20)

cleaned_data.to_csv('./cleaned_data.csv', index=False)

# Split data into multiple dataframes, one per student.
# Each student's dataframe contains all the questions they answered for all concepts.
grouped_by_sid = cleaned_data.groupby(['student_id'])

43

In [13]:
# Since each student's dataframe contains questions from all concepts, we use 
# pandas groupby again to create one dataframe per concept per student.
# If there are k students and j concepts, we end up with k*j dataframes.
# The kc_ktracedskills column contains the concept.

concept_by_students = {}
for name,group in grouped_by_sid:
    concept_by_students[name] = group.groupby(['kc_ktracedskills'])

43


In [18]:
# Create a sequence of answers as 1's and 0's for each student and each concept
# For each question (row in data), there are two columns of interest:
# 1. corrects - This is 1 if the student got the answer correct on the first try, 0 otherwise
# 2. incorrects - This is 0 if the student got the answer correct on the first try, otherwise it is >= 1
#                 This implies that the student can attempt the same question multiple times and keep getting it wrong.
# There are two ways of turning this data into a sequence of corrects (1's) and incorrects (0's).
# 1. We only keep track of their first answer and disregard subsequent attempts
# 2. We count all incorrect attempts as 0's
# If we set first_attempt_only = True in this function, we use method 1 to create the sequence of answers.
# If it's false, we use method 2.

def create_student_answer_sequences(concept_by_students, first_attempt_only=True):
    list_alldata = {}

    for student, concept in concept_by_students.iteritems():
        concepts = {}
        for name,group in concept:
            # We are looking at concept with name where group is df containing all questions
            # student answered on that concept.

            answers = []
            for index, row in group.iterrows():
                if first_attempt_only == False:
                    num_attempts = int(row['incorrects']) + int(row['corrects'])
                    corrects = int(row['corrects'])
                    temp_ans = [0 for i in range(num_attempts - corrects)] + [1 for i in range(corrects)]
                else:
                    temp_ans = [row['correct_first_attempt']]
                if name in concepts:
                    concepts[name] += temp_ans
                else:
                    concepts[name] = temp_ans
        list_alldata[student] = concepts

    return list_alldata

list_alldata = create_student_answer_sequences(concept_by_students)

In [22]:
# Combine sequences of ansewrs as 1's and 0's by concept, where all students'
# answers are in a list of lists

from collections import defaultdict
final_dict = defaultdict(dict)
for student, answers in list_alldata.iteritems():
    for concept, sequence in answers.iteritems():
        # Clean concept names so matlab can read them
        new_concept = concept.replace(' ', '_')
        new_concept = new_concept.lower()
        new_concept = "".join([ c if c.isalnum() else "_" for c in new_concept ])        
        final_dict[new_concept][student] = sequence
        
sio.savemat('../matlab/final_dict.mat', final_dict)

In [70]:
# Format the data so matlab can read it in as a cell array
final_dict_np = {}
for k,v in final_dict.iteritems():
    obj_arr = np.zeros((len(v)), dtype=np.object)
    for seq_i in range(len(v)):
        obj_arr[seq_i] = v[seq_i]
    final_dict_np[k] = obj_arr
sio.savemat('../matlab/final_dict.mat', final_dict_np)